In [1]:
import numpy as np
import tensorflow as tf
import pickle

# 讀取 Muist 數字手寫資料 pickle
with open('mnist.pickle', 'rb') as file:
    (x_train, t_train), (x_test, t_test) = pickle.load(file)  

from PIL import Image
    
def img_show(img):
    return Image.fromarray(np.uint8(img*255))

In [2]:
def create_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer' + n_layer
    # W & b 初始化條件 init
    W_init = tf.random_normal_initializer(stddev=(2.0/in_size)**0.5)
    b_init = tf.zeros_initializer()
    # W & b 變數設定
    Weights = tf.get_variable(name=layer_name+'_W', shape=[in_size, out_size], initializer=W_init)
    tf.summary.histogram('weights', Weights) # 紀錄資料
    biases = tf.get_variable(name=layer_name+'_b', shape=[out_size], initializer=b_init)
    tf.summary.histogram('biases', biases) # 紀錄資料
    Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    # 激活函數設定
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    tf.summary.histogram('outputs', outputs)
    
    return outputs

In [3]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 784], name='x_input')
    ts = tf.placeholder(tf.float32, [None, 10], name='t_input')
    keep_prob = tf.placeholder(tf.float32)
    
with tf.variable_scope('layerA'):
    layerAA = create_layer(xs, 784, 256, n_layer='A', activation_function=tf.nn.relu)
    layerA = tf.nn.dropout(layerAA, keep_prob=0.7)
    
with tf.variable_scope('layerB'):
    layerBB = create_layer(layerA, 256, 128, n_layer='B', activation_function=tf.nn.relu)
    layerB = tf.nn.dropout(layerBB, keep_prob=0.7)
    
with tf.variable_scope('layerC'):
    layerC = create_layer(layerB, 128, 10, n_layer='C', activation_function=tf.nn.relu)
    
y = tf.nn.softmax(layerC)

# cross entropy
with tf.name_scope('loss'):
    loss = -tf.reduce_mean(tf.reduce_sum((ts * tf.log(tf.clip_by_value(y, 1e-16, 1))), axis=1)) # 輸入為 batch 使用平均值 mean
    tf.summary.scalar('loss', loss) # 紀錄資料

#train_way = tf.train.GradientDescentOptimizer(learning_rate=0.01) # Acc : 0.9802  (learning_rate=0.01
train_way = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.92) # Acc : 0.9829  (learning_rate=0.01, momentum=0.92)

with tf.name_scope('train'):
    train_step = train_way.minimize(loss)

with tf.name_scope('accuracy'):
    # tf.cast() 改變資料型態 np.astype change dtype
    acc = tf.cast(tf.equal(tf.argmax(y, axis=1), tf.argmax(ts, axis=1)), tf.float32)
    accuracy = tf.reduce_mean(acc, name='accuracy')
    tf.summary.scalar('accuracy', accuracy) # 紀錄資料

In [4]:
sess = tf.Session()
merged = tf.summary.merge_all() # 所以紀錄資料設定
writer = tf.summary.FileWriter("logs/", sess.graph) # 紀錄圖形
sess.run(tf.global_variables_initializer()) # 變數初始化

In [5]:
data_num = x_train.shape[0]
batch = 100
lst = np.arange(data_num)
feed_all = {xs: x_train, ts: t_train}
for i_ in range(10):
    # 隨機打亂資料排序
    np.random.shuffle(lst)
    x_train = x_train[lst]
    t_train = t_train[lst]
   
    for i in range(0, data_num, batch):  # 每 bench 100個資料
        feed_data = {xs: x_train[i:i + batch], ts: t_train[i:i + batch]}
        sess.run(train_step, feed_dict=feed_data)
        
    result = sess.run(merged, feed_dict=feed_all) # 寫入紀錄
    writer.add_summary(result, i_)
    
    # Accuracy test data
    acc_test = sess.run(accuracy, feed_dict={xs: x_test, ts: t_test})
    acc_train = sess.run(accuracy, feed_dict=feed_all)
    print('%3d  train:%.4f  test:%.4f'%(i_, acc_train, acc_test))
    
# in win10 open powershell key:  tensorboard --logdir="logs" 

  0  train:0.9178  test:0.9184
  1  train:0.9416  test:0.9374
  2  train:0.9507  test:0.9483
  3  train:0.9597  test:0.9565
  4  train:0.9644  test:0.9581
  5  train:0.9669  test:0.9622
  6  train:0.9691  test:0.9608
  7  train:0.9722  test:0.9656
  8  train:0.9734  test:0.9628
  9  train:0.9757  test:0.9676


In [6]:
#sess.close()

# in win10 open powershell key:  tensorboard --logdir="logs" 